In [1]:
import torch
from torch import nn

import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [3]:
train_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=None
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

100%|██████████| 26421880/26421880 [00:29<00:00, 891463.98it/s] 


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 168504.07it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3048456.76it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 8633457.41it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [4]:
class_names = train_data.classes
class_names

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

In [5]:
BATCH_SIZE = 32

train_dataloader = DataLoader(train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

test_dataloader = DataLoader(test_data,
                             batch_size=BATCH_SIZE,
                             shuffle=False)

In [49]:
class FashionMNISTModel(nn.Module):

  def __init__(self, input_size, output_size):
    super().__init__()
    self.block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_size,
                  out_channels=8,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=8,
                  out_channels=8,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )
    self.block_2 = nn.Sequential(
        nn.Conv2d(in_channels=8,
                  out_channels=8,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=8,
                  out_channels=8,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=392,
                  out_features=output_size)
    )

  def forward(self, x):
    x = self.block_1(x)
    # print(x.shape)
    x = self.block_2(x)
    # print(x.shape)
    x = self.classifier(x)
    # print(x.shape)

    return x

In [50]:
model = FashionMNISTModel(input_size=1, output_size=len(class_names)).to(device)
model

FashionMNISTModel(
  (block_1): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=392, out_features=10, bias=True)
  )
)

In [51]:
image_tensor = torch.rand(32, 1, 28, 28).to(device)
model(image_tensor)

tensor([[-0.0040, -0.0162,  0.0034,  0.0876,  0.0122,  0.0500,  0.0170, -0.0648,
          0.0416, -0.0840],
        [-0.0031, -0.0103,  0.0041,  0.0892,  0.0139,  0.0460,  0.0186, -0.0684,
          0.0414, -0.0858],
        [-0.0047, -0.0165,  0.0048,  0.0882,  0.0149,  0.0429,  0.0178, -0.0636,
          0.0443, -0.0873],
        [-0.0060, -0.0152,  0.0037,  0.0899,  0.0126,  0.0456,  0.0192, -0.0663,
          0.0393, -0.0854],
        [-0.0067, -0.0172,  0.0048,  0.0858,  0.0140,  0.0423,  0.0186, -0.0666,
          0.0397, -0.0883],
        [-0.0041, -0.0164,  0.0065,  0.0888,  0.0127,  0.0455,  0.0170, -0.0661,
          0.0409, -0.0856],
        [-0.0048, -0.0136,  0.0052,  0.0861,  0.0113,  0.0446,  0.0226, -0.0643,
          0.0431, -0.0857],
        [-0.0060, -0.0161,  0.0058,  0.0877,  0.0112,  0.0505,  0.0165, -0.0665,
          0.0430, -0.0854],
        [-0.0039, -0.0146,  0.0054,  0.0885,  0.0134,  0.0444,  0.0163, -0.0660,
          0.0410, -0.0839],
        [-0.0069, -

In [52]:
next(iter(train_dataloader))[0].shape

torch.Size([32, 1, 28, 28])

In [53]:
def accuracy_fn(y_true, y_preds):
  return (y_true == y_preds).sum().item() / len(y_preds)

In [54]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(),
                             lr=1e-3)

In [55]:
N_EPOCHS = 3

for epoch in tqdm(range(N_EPOCHS)):

  train_loss = 0
  train_acc = 0

  for batch, (X, y) in enumerate(train_dataloader):

    X, y = X.to(device), y.to(device)

    model.train()

    y_logits = model(X)
    loss = loss_fn(y_logits, y)
    train_loss += loss
    train_acc += accuracy_fn(y, y_logits.argmax(dim=1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  train_loss /= len(train_dataloader)
  train_acc /= len(train_dataloader)

  test_loss, test_acc = 0, 0

  model.eval()
  with torch.inference_mode():
    for X, y in test_dataloader:

      X, y = X.to(device), y.to(device)
      test_logits = model(X)
      test_loss += loss_fn(test_logits, y)
      test_acc += accuracy_fn(y_true=y, y_preds=test_logits.argmax(dim=1))

    test_loss /= len(test_dataloader)
    test_acc /= len(test_dataloader)

  print(f'Train Loss: {train_loss:.2f} | Train Accuracy: {train_acc:.2f} | Test Loss: {test_loss:.2f} | Test Accuracy: {test_acc:.2f}')

  0%|          | 0/3 [00:00<?, ?it/s]

Train Loss: 0.53 | Train Accuracy: 0.81 | Test Loss: 0.41 | Test Accuracy: 0.85
Train Loss: 0.36 | Train Accuracy: 0.87 | Test Loss: 0.36 | Test Accuracy: 0.87
Train Loss: 0.33 | Train Accuracy: 0.88 | Test Loss: 0.34 | Test Accuracy: 0.88


In [56]:
# image_batch, image_labels = next(iter(train_dataloader))

In [57]:
def eval_model(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               device: torch.device,
               accuracy_fn):
  loss, acc = 0, 0
  model.eval()
  with torch.inference_mode():
    for X, y in data_loader:
      X, y = X.to(device), y.to(device)
      y_logits = model(X)
      loss += loss_fn(y_logits, y)
      acc += accuracy_fn(y_true=y, y_preds=y_logits.argmax(dim=1))

    loss /= len(data_loader)
    acc /= len(data_loader)

  return {'model_name': model.__class__.__name__,
          'model_loss': loss.item(),
          'model_acc': acc}

In [58]:
eval_model(model=model,
           data_loader=test_dataloader,
           loss_fn=nn.CrossEntropyLoss(),
           device=device,
           accuracy_fn=accuracy_fn)

{'model_name': 'FashionMNISTModel',
 'model_loss': 0.3414200246334076,
 'model_acc': 0.876797124600639}